In [20]:
from typing import List, Optional
from pydantic import BaseModel
from google import genai
from store import VectorStore 
from exa_py import Exa
import os

def get_rephraser_prompt(
    question: str,
    local_observations: Optional[List[str]] = [],
    web_observations: Optional[List[str]] = [],
    previous_queries: Optional[List[str]] = [],
) -> str:
    if len(local_observations)==0 and len(web_observations)==0:
        template = f"""You are an assistant tasked with taking a natural languge query from a user
and converting it into a query for a vectorstore. These queries are used to do deep research on a topic provided. In the process, strip out all 
information that is not relevant for the retrieval task and return a new, simplified
question for vectorstore retrieval. Always provide the respone in the given JSON format

Write the queries such that it explores different perspectives of the of topics related to the user query. Give 3 different queries.
Here is the user query: {question}

Set the done flag to False
"""
    else:
        local_observations = "\n".join(local_observations)            
        if len(web_observations) > 0:
            web_observations = "\n".join(web_observations)
            template = f"""You are an assistant tasked with taking a natural languge query from a user
and converting it into a query for a vectorstore. In the process, strip out all 
information that is not relevant for the retrieval task and return a new, simplified
question for vectorstore retrieval. 

Here is the user query that the user wants to do deep research on.

User query: {question}

The following information is already collected. The information within the <local_observation> tag is collected from the local documents that the user has. The infomation withing the <web_observations> tag is collected from the web. 
The previously used queries are also provided within the <previous_queries> tag. Make sure to provide more queries that explore different perspectives of the topic.  

You should give more focus on the local observations and use the web observations only if needed. 

<previous_queries>
{previous_queries}
</previous_queries>

<local_observations>
{local_observations}
</local_observations>

<web observations>
{web_observations}
</web_observations>


Based on this observations, you have two options:
1. Find knowledge gaps that still need to be explored and write 3 different queries that explore different perspectives of the topic. If this is the case set the done flag to False.
2. If there are no more knowledge gaps and you have enough information related to the topic, you dont have to provide any more queries and you can set the done flag to True. 

Before setting the done flag to true, make sure that the following conditions are met: 
1. You have explored different perspectives of the topic
2. You have collected some opposing views
3. You have collected some supporting views
4. You have collected some views that are not directly related to the topic but can be used to explore the topic further.

Always provide the respone in the given JSON format

    """
        else:
            template = f"""You are an assistant tasked with taking a natural languge query from a user
and converting it into a query for a vectorstore. In the process, strip out all 
information that is not relevant for the retrieval task and return a new, simplified
question for vectorstore retrieval. 

Here is the user query that the user wants to do deep research on.

User query: {question}

The following information is already collected. The information within the <local_observation> tag is collected from the local documents that the user has.
The previously used queries are also provided within the <previous_queries> tag. Make sure to provide more queries that explore different perspectives of the topic.  

<previous_queries>
{previous_queries}
</previous_queries>

<local_observations>
{local_observations}
</local_observations>


Based on this observations, you have two options:
1. Find knowledge gaps that still need to be explored and write 3 different queries that explore different perspectives of the topic. If this is the case set the done flag to False.
2. If there are no more knowledge gaps and you have enough information related to the topic, you dont have to provide any more queries and you can set the done flag to True. 

Before setting the done flag to true, make sure that the following conditions are met: 
1. You have explored different perspectives of the topic
2. You have collected some opposing views
3. You have collected some supporting views
4. You have collected some views that are not directly related to the topic but can be used to explore the topic further.

Always provide the respone in the given JSON format

    """
    return template

In [ ]:
client = genai.Client()

if os.path.exists("doc"):
    my_store =  VectorStore("doc")
else:
    os.mkdir("doc")
    my_store =  VectorStore("doc")
exa = Exa(api_key=os.environ.get("EXA_API_KEY"))

web_search  = False

class QueryRephase(BaseModel):
    content: str
    querys: list[str]
    done: bool

question = "what are the differences between SSM models and transformer models?"


def rephrase(prompt: str) -> list:
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': QueryRephase,
        },
    )
    return response.text

def query_response(prompt: str) -> QueryRephase:
    response = rephrase(prompt)
    my_recipes= str(response)
    out = QueryRephase.model_validate_json(my_recipes)
    return out

def report_response(prompt: str) -> str:
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt,
    )
    return response.text

def get_observations(queries: List[str]) -> List[str]:
    local_observations = []
    web_observations = []
    for query in queries:
        local_observation = my_store.forward(query)
        local_observations.extend(list(local_observation))

        if web_search:
            web_result = exa.search_and_contents(query, type="auto", text=True, num_results=3)
            web_observation = []
            for result in web_result.results:
                observation = f"Title: {result.title} \n URL: {result.url} \n Content: {result.text}"
                web_observation.append(observation)
            web_observations.extend(web_observation)
    
    return local_observations, web_observations

In [22]:
def do_research(question: str, max_steps: int = 5) -> List[str]:
    local_observations, web_observations, previous_queries = [], [], []
    for i in range(max_steps):
        print("Step Number: ", i)
        prompt = get_rephraser_prompt(question, local_observations, web_observations, previous_queries)
        query_responses = query_response(prompt)
        print("Searching with queries: ", "\n".join(query_responses.querys))
        print("Done: ", query_responses.done)
        if query_responses.done:
            break
        local_observation, web_observation= get_observations(query_responses.querys)
        if len(local_observations)==0 and len(web_observations)==0:
            local_observations = local_observation
            web_observations = web_observation
        else:
            local_observations.extend(local_observation)
            web_observations.extend(web_observation)
    return local_observations, web_observations


In [23]:
local_observations, web_observations = do_research(question)

Step Number:  0
Searching with queries:  What are the architectural differences between State Space Models (SSMs) and Transformer models in deep learning?
How do SSMs and Transformers compare in terms of computational complexity, training efficiency, and memory requirements?
In what specific applications or tasks do SSMs outperform Transformers, and vice versa, highlighting the trade-offs between the two?
Done:  False
Step Number:  1
Searching with queries:  Advantages of selective state space models (SSMs) over traditional SSMs.
How do SSMs address the computational inefficiency of Transformers on long sequences?
What are the key weaknesses of non-selective SSMs compared to Transformers?
Done:  False
Step Number:  2
Searching with queries:  
Done:  True


In [24]:
def write_report(question: str, local_observations: List[str], web_observations: List[str]) -> None:
    if len(web_observations) > 0:
        template = f"""You are an expert in writing reports. You are provided with the user query and the collected information from both local and web sources. 
The local information is within the <local_observations> tag and the web information is within the <web_observations> tag.

User query: {question}

<local_observations>
{local_observations}
</local_observations>

<web_observations>
{web_observations}
</web_observations>

Write a detailed report on the user query based on the information provided. You should provide a detailed analysis of the topic and provide a summary of the information collected from both local and web sources

Instructions:
1. Write a detailed report on the user query based on the information provided. You should provide a detailed analysis of the topic and provide a summary of the information collected from both local and web sources
2. Format the report in a way that is easy to read and understand
3. Do not explicitly mention if the output is from local or web observations. Just write the report as if you have all the information available.
4. Structure the report with an introduction, body and conclusion
5. Provide inline citations if needed. Cite the file name for the local observations and the URL for the web observations. Provide all references at the end of the report.
6. Provide tables if needed to show differences

        """

    if len(web_observations) == 0:
        template = f"""You are an expert in writing reports. You are provided with the user query and the collected information from local sources. 
The local information is within the <local_observations> tag.

User query: {question}

<local_observations>
{local_observations}
</local_observations>

Instructions:
1. Write a detailed report on the user query based on the information provided. You should provide a detailed analysis of the topic and provide a summary of the information collected from both local and web sources
2. Format the report in a way that is easy to read and understand
3. Do not explicitly mention if the output is from local or web observations. Just write the report as if you have all the information available.
4. Structure the report with an introduction, body and conclusion
5. Provide inline citations if needed. Cite the file name. Provide all references as file name at the end of the report.
6. Provide tables if needed to show differences
        """

    response = report_response(template)
    return response

    

In [25]:
report = write_report(question, local_observations, web_observations)

with open("report.md", "w") as f:
    f.write(report)